# LLM API Demo (MASSafetyGuard)

This notebook calls the same LLM backend used by the MAS tests by loading `config/mas_llm_config.yaml` via `get_mas_llm_config()`.
If this runs successfully, the LLM connectivity for the MAS test scripts should also be OK.

In [1]:
import sys
from pathlib import Path

# Find repo root (folder containing 'src')
project_root = Path.cwd().resolve()
while not (project_root / "src").exists() and project_root != project_root.parent:
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.utils.llm_config import get_mas_llm_config
from src.utils.llm_client import get_llm_client

cfg = get_mas_llm_config()
print("provider:", cfg.provider)
print("model:", cfg.model)
print("base_url:", cfg.base_url)
try:
    _ = cfg.get_api_key()
    print("api_key: loaded")
except Exception as e:
    print("api_key: error ->", e)

client = get_llm_client(config=cfg)
response = client.generate("Reply with OK only.", max_tokens=20)
print("response:", response)


provider: openai
model: gpt-4o-mini
base_url: http://35.220.164.252:3888/v1/
api_key: loaded
response: OK


In [2]:
from src.utils.llm_config import get_monitor_llm_config
from src.utils.llm_client import get_monitor_llm_client

cfg = get_monitor_llm_config()
print(cfg.provider, cfg.model, cfg.base_url)
print("api_key:", "ok" if cfg.get_api_key() else "missing")
  
client = get_monitor_llm_client()
print(client.generate_with_system("You are a tester.", "Reply OK only.", max_tokens=20))

openai gpt-4o-mini http://35.220.164.252:3888/v1/
api_key: ok
OK


In [2]:
import sys
import os
from pathlib import Path

# Add project root to sys.path to allow importing from src
project_root = Path.cwd().resolve()
while not (project_root / "src").exists() and project_root != project_root.parent:
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Test GPT-5 Compatibility
from src.utils.llm_config import get_mas_llm_config
from openai import OpenAI

config = get_mas_llm_config()
print(f"Testing with model: gpt-5 (overriding yaml config for this cell)")

client = OpenAI(
    api_key=config.get_api_key(),
    base_url=config.base_url
)

try:
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "user", "content": "Hello! Are you GPT-5?"}
        ],
        temperature=0.7,
        max_tokens=2000
    )
    print("\nSuccess! Response received:")
    print("-" * 20)
    print("Response content (repr):")
    print(repr(response.choices[0].message.content))
    print(f"Finish reason: {response.choices[0].finish_reason}")
    print("-" * 20)
    print(f"Model used: {response.model}")
except Exception as e:
    print("\nError occurred calling the API:")
    print(f"{e}")

Testing with model: gpt-5 (overriding yaml config for this cell)

Success! Response received:
--------------------
Response content (repr):
'Hello! I’m ChatGPT, an OpenAI model from the GPT‑4 family—not GPT‑5. How can I help you today?'
Finish reason: stop
--------------------
Model used: gpt-5-2025-08-07
